# I decided to switch to using raw FF output for unet segmentation training. This script replaces all processed training files with raw

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from src.functions.utilities import path_leaf
pyo.init_notebook_mode()

### Get list of unet training directories

In [ ]:
from glob2 import glob

root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"
unet_root = os.path.join(root, "built_image_data", "unet_training", "*")
unet_dirs = sorted(glob(unet_root))
unet_dirs = [u for u in unet_dirs if os.path.isdir(u)]

### iterate through each one and compile list of unique labels

In [ ]:
import shutil
from tqdm import tqdm

image_root = os.path.join(root, "built_image_data", "stitched_FF_images")

for u, udir in enumerate(tqdm(unet_dirs)):
    # initialize list  to store label files
    label_path_list = []
    label_name_list = []
    # get list of subdirectories
    subdir_list = sorted(glob(os.path.join(udir, "*")))
    subdir_list = [s for s in subdir_list if os.path.isdir(s)]
    subdir_list = [s for s in subdir_list if "training" not in path_leaf(s)]
    for s, sdir in enumerate(subdir_list):
        a_dir = os.path.join(sdir, "annotations","")
        if os.path.isdir(a_dir):
            lb_files = glob(a_dir + "*.tif") + glob(a_dir + "*.jpg")
            lb_names = [path_leaf(im) for im in lb_files]
            label_path_list += lb_files
            label_name_list += lb_names

    # make new directory
    ann_dir = os.path.join(udir, "training", "annotations", "")
    if not os.path.isdir(ann_dir):
        os.makedirs(ann_dir)

    # get de-duped list of images
    lb_names_u, si_u = np.unique(label_name_list, return_index=True) 
    lb_paths_u = [label_path_list[i] for i in si_u]

    # copy over
    for i in range(len(lb_names_u)):
        shutil.copyfile(lb_paths_u[i], os.path.join(ann_dir, lb_names_u[i]))

    # make image directory
    im_dir = os.path.join(udir, "training", "images", "")
    if not os.path.isdir(im_dir):
        os.makedirs(im_dir)

    # copy images
    for i in range(len(lb_names_u)):
        date_folder = lb_names_u[i][0:8]
        im_stub = lb_names_u[i][9:18]
        im_path_from = glob(os.path.join(image_root, date_folder, im_stub + "*"))[0]
        new_name = path_leaf(im_path_from)
        im_path_to = os.path.join(im_dir, date_folder + "_" + new_name)
        shutil.copyfile(im_path_from, im_path_to)
    
    
print(label_path_list[0])

### Split the live and dead labels into seperate sets

In [ ]:
import skimage.io as io

emb_dir = os.path.join(unet_root[:-1], "UNET_training_emb", "training") 
via_dir_lb = os.path.join(unet_root[:-1], "UNET_training_via", "training", "annotations") 
via_dir_im = os.path.join(unet_root[:-1], "UNET_training_via", "training", "images") 
if not os.path.isdir(os.path.join(via_dir_im)):
    os.makedirs(via_dir_lb)
    os.makedirs(via_dir_im)
    
mask_dir_lb = os.path.join(unet_root[:-1], "UNET_training_mask", "training", "annotations")
mask_dir_im = os.path.join(unet_root[:-1], "UNET_training_mask", "training", "images")
if not os.path.isdir(os.path.join(mask_dir_im)):
    os.makedirs(mask_dir_lb)
    os.makedirs(mask_dir_im)

label_list = sorted(glob(os.path.join(emb_dir, "annotations", "*")))
image_list = sorted(glob(os.path.join(emb_dir, "images", "*")))

for i in tqdm(range(len(image_list))):

    image_name = path_leaf(image_list[i])
    anno_path = glob(os.path.join(emb_dir, "annotations", image_name[0:19] + "*"))
    # loa
    im = io.imread(image_list[i])

    if len(anno_path) >0:
        lb = io.imread(anno_path[0])
         # make alternative labels
        lb_mask = lb.copy()
        lb_mask[lb_mask==2] = 1
    
        lb_via = lb.copy()
        lb_via[lb_via==1] = 0
        lb_via[lb_via==2] = 1

        io.imsave(os.path.join(mask_dir_lb, image_name), lb_mask, check_contrast=False)
        io.imsave(os.path.join(via_dir_lb, image_name), lb_via, check_contrast=False)
        
    # save
    io.imsave(os.path.join(mask_dir_im, image_name), im, check_contrast=False)
    io.imsave(os.path.join(via_dir_im, image_name), im, check_contrast=False)

In [ ]:
/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/built_image_data/unet_training/UNET_training_emb/training/annotations/